Import Basic python libraries

In [ ]:
from pyspark.sql.functions import col, explode, sum, max, min, avg, count, lit
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType
from pyspark.sql.window import Window

1. set the variables for credentials for kafka

In [ ]:
# Credentials and configuration
clustername='***'
confluentBootstrapServers = '*****************************************************'
confluentApiKey = '********************'
confluentSecret = '*****************************************************'
confluentTopicName = '********************'

2. build a connection between Confluent kafka and Databricks

In [ ]:
growth_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "confluentBootstrapServers") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option(f"kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username={confluentApiKey} password={confluentSecret};") \
    .option("kafka.ssl.endpoint.identification.algorithm", "https") \
    .option("subscribe", confluentTopicName) \
    .option("maxOffsetsPerTrigger", 50) \
    .load()



3. varify the incoming format of kafka messeges

In [ ]:
display(growth_df)

key,value,topic,partition,offset,timestamp,timestampType
MTA1,eyJHZW5vdHlwZU5hbWUiOjEwNSwicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjozMS4wNCwicGxhbnRfaGVpZ2h0XzYwREFUX0NNIjoyOC4xMywicGxhbnRfaGVpZ2h0XzkwREFUX0NNIjozNy44MywiZGF5c19mbG93ZXJfaW5pdGk= (truncated),growth_data_stream,5,46,2024-07-04T14:18:40.062+0000,0
MTA2,eyJHZW5vdHlwZU5hbWUiOjEwNiwicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjozMS42OCwicGxhbnRfaGVpZ2h0XzYwREFUX0NNIjoyOC43MSwicGxhbnRfaGVpZ2h0XzkwREFUX0NNIjozOC42MSwiZGF5c19mbG93ZXJfaW5pdGk= (truncated),growth_data_stream,3,47,2024-07-04T14:18:44.956+0000,0
MTA3,eyJHZW5vdHlwZU5hbWUiOjEwNywicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjo1Mi45MiwicGxhbnRfaGVpZ2h0XzYwREFUX0NNIjozOS4yLCJwbGFudF9oZWlnaHRfOTBEQVRfQ00iOjU0Ljg4LCJkYXlzX2Zsb3dlcl9pbml0aWE= (truncated),growth_data_stream,1,60,2024-07-04T14:18:45.501+0000,0
MTA4,eyJHZW5vdHlwZU5hbWUiOjEwOCwicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjo1My40NiwicGxhbnRfaGVpZ2h0XzYwREFUX0NNIjozOS42LCJwbGFudF9oZWlnaHRfOTBEQVRfQ00iOjU1LjQ0LCJkYXlzX2Zsb3dlcl9pbml0aWE= (truncated),growth_data_stream,4,48,2024-07-04T14:18:52.903+0000,0
MTA5,eyJHZW5vdHlwZU5hbWUiOjEwOSwicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjo1NS42MiwicGxhbnRfaGVpZ2h0XzYwREFUX0NNIjo0MS4yLCJwbGFudF9oZWlnaHRfOTBEQVRfQ00iOjU3LjY4LCJkYXlzX2Zsb3dlcl9pbml0aWE= (truncated),growth_data_stream,4,49,2024-07-04T14:18:55.297+0000,0
MTEy,eyJHZW5vdHlwZU5hbWUiOjExMiwicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjo1Ni4xLCJwbGFudF9oZWlnaHRfNjBEQVRfQ00iOjM5Ljc4LCJwbGFudF9oZWlnaHRfOTBEQVRfQ00iOjQ5Ljk4LCJkYXlzX2Zsb3dlcl9pbml0aWE= (truncated),growth_data_stream,1,61,2024-07-04T14:18:56.963+0000,0
MTEw,eyJHZW5vdHlwZU5hbWUiOjExMCwicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjo1Mi4yNSwicGxhbnRfaGVpZ2h0XzYwREFUX0NNIjozNy4wNSwicGxhbnRfaGVpZ2h0XzkwREFUX0NNIjo0Ni41NSwiZGF5c19mbG93ZXJfaW5pdGk= (truncated),growth_data_stream,5,47,2024-07-04T14:18:55.876+0000,0
MTEx,eyJHZW5vdHlwZU5hbWUiOjExMSwicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjo1Ni42NSwicGxhbnRfaGVpZ2h0XzYwREFUX0NNIjo0MC4xNywicGxhbnRfaGVpZ2h0XzkwREFUX0NNIjo1MC40NywiZGF5c19mbG93ZXJfaW5pdGk= (truncated),growth_data_stream,5,48,2024-07-04T14:18:56.428+0000,0
MTEz,eyJHZW5vdHlwZU5hbWUiOjExMywicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjozNS43LCJwbGFudF9oZWlnaHRfNjBEQVRfQ00iOjM5Ljc4LCJwbGFudF9oZWlnaHRfOTBEQVRfQ00iOjQ4Ljk2LCJkYXlzX2Zsb3dlcl9pbml0aWE= (truncated),growth_data_stream,5,49,2024-07-04T14:18:57.578+0000,0
MTE0,eyJHZW5vdHlwZU5hbWUiOjExNCwicGxhbnRfaGVpZ2h0XzMwREFUX0NNIjozMy42LCJwbGFudF9oZWlnaHRfNjBEQVRfQ00iOjM3LjQ0LCJwbGFudF9oZWlnaHRfOTBEQVRfQ00iOjQ2LjA4LCJkYXlzX2Zsb3dlcl9pbml0aWE= (truncated),growth_data_stream,0,38,2024-07-04T14:18:58.143+0000,0


4. deserialized the key:value pairs of kafka messeges

In [ ]:
desrialised_df = growth_df.selectExpr("cast(key as string) as key", "cast (value as string) as value", "topic", "partition", "offset", "timestamp", "timestampType")

5. confirm your streaming data once desrialised

In [ ]:
display(desrialised_df)

key,value,topic,partition,offset,timestamp,timestampType
101,"{""GenotypeName"":101,""plant_height_30DAT_CM"":47.47,""plant_height_60DAT_CM"":52.52,""plant_height_90DAT_CM"":63.63,""days_flower_initiation_DAYS"":35.19,""nu_primery_branches_Num"":11.39,""fruit_length_cm"":6.21,""fruit_circumference_mm"":12.63,""avg_fruit_weight_gram"":33.89,""nu_fruits_per_plant"":6.06,""fruit_TSS_Brix"":5.95,""fruit_yield_per_plant_Kg"":203.34,""yield_status"":""Medium"",""morpho_details"":[{""StemAnthocyanin"":""Absent"",""AnthocyaninIntensity"":""Absent"",""StemPubescence"":""Present"",""StemPubescenceIntensity"":""Medium"",""LeafBladeColour"":""Green"",""LeafBladeColourIntensity"":""Dark"",""ColourOfVein"":""Green"",""IntensityOfColour8"":""Medium"",""SpineonLeaf"":""Absent"",""FlowerColour"":""Purple"",""FruitingPattern"":""Cluster"",""FruitColour"":""Purple"",""IntensityofColour13"":""Medium"",""Stripes"":""Present"",""StripesDensity"":""Sparse"",""FruitPatches"":""Present"",""SpinesInCalyx"":""Absent"",""DensityOfSpininess"":""Absent"",""FruitShape"":""Obovate""}]}",growth_data_stream,2,0,2024-07-04T13:03:12.185+0000,0
120,"{""GenotypeName"":120,""plant_height_30DAT_CM"":72.1,""plant_height_60DAT_CM"":61.8,""plant_height_90DAT_CM"":74.16,""days_flower_initiation_DAYS"":45.6,""nu_primery_branches_Num"":6.72,""fruit_length_cm"":5.7,""fruit_circumference_mm"":11.31,""avg_fruit_weight_gram"":38.43,""nu_fruits_per_plant"":6.18,""fruit_TSS_Brix"":4.75,""fruit_yield_per_plant_Kg"":230.58,""yield_status"":""Medium"",""morpho_details"":[{""StemAnthocyanin"":""Absent"",""AnthocyaninIntensity"":""Absent"",""StemPubescence"":""Present"",""StemPubescenceIntensity"":""Medium"",""LeafBladeColour"":""Green"",""LeafBladeColourIntensity"":""Dark"",""ColourOfVein"":""Green"",""IntensityOfColour8"":""Medium"",""SpineonLeaf"":""Absent"",""FlowerColour"":""Dark _Purple"",""FruitingPattern"":""Cluster"",""FruitColour"":""Green"",""IntensityofColour13"":""Medium"",""Stripes"":""Absent"",""StripesDensity"":""Absent"",""FruitPatches"":""Present"",""SpinesInCalyx"":""Present"",""DensityOfSpininess"":""Med ium"",""FruitShape"":""Globular""}]}",growth_data_stream,2,1,2024-07-04T13:03:47.276+0000,0
121,"{""GenotypeName"":121,""plant_height_30DAT_CM"":70.7,""plant_height_60DAT_CM"":60.6,""plant_height_90DAT_CM"":72.72,""days_flower_initiation_DAYS"":48.72,""nu_primery_branches_Num"":7.18,""fruit_length_cm"":6.09,""fruit_circumference_mm"":12.08,""avg_fruit_weight_gram"":41.06,""nu_fruits_per_plant"":6.06,""fruit_TSS_Brix"":5.08,""fruit_yield_per_plant_Kg"":246.36,""yield_status"":""Medium"",""morpho_details"":[{""StemAnthocyanin"":""Absent"",""AnthocyaninIntensity"":""Absent"",""StemPubescence"":""Present"",""StemPubescenceIntensity"":""Strong"",""LeafBladeColour"":""Green"",""LeafBladeColourIntensity"":""Medium"",""ColourOfVein"":""Green"",""IntensityOfColour8"":""Medium"",""SpineonLeaf"":""Absent"",""FlowerColour"":""Light_Purple"",""FruitingPattern"":""Solitary"",""FruitColour"":""Purple"",""IntensityofColour13"":""Dark"",""Stripes"":""Absent"",""StripesDensity"":""Absent"",""FruitPatches"":""Absent"",""SpinesInCalyx"":""Present"",""DensityOfSpininess"":""Medium"",""FruitShape"":""Obovate""}]}",growth_data_stream,2,2,2024-07-04T13:03:47.896+0000,0
122,"{""GenotypeName"":122,""plant_height_30DAT_CM"":49.98,""plant_height_60DAT_CM"":52.02,""plant_height_90DAT_CM"":64.26,""days_flower_initiation_DAYS"":53.82,""nu_primery_branches_Num"":7.32,""fruit_length_cm"":5.8,""fruit_circumference_mm"":15.53,""avg_fruit_weight_gram"":58.37,""nu_fruits_per_plant"":5.1,""fruit_TSS_Brix"":4.4,""fruit_yield_per_plant_Kg"":291.87,""yield_status"":""High"",""morpho_details"":[{""StemAnthocyanin"":""Present"",""AnthocyaninIntensity"":""Medium"",""StemPubescence"":""Present"",""StemPubescenceIntensity"":""Medium"",""LeafBladeColour"":""Green"",""LeafBladeColourIntensity"":""Dark"",""ColourOfVein"":""Purple"",""IntensityOfColour8"":""Medium"",""SpineonLeaf"":""Absent"",""FlowerColour"":""

6. specify the schema for actual data which is present in value column of kafka reads

In [ ]:
schema = StructType([
    StructField("GenotypeName", IntegerType()),
    StructField("plant_height_30DAT_CM", DoubleType()),
    StructField("plant_height_60DAT_CM", DoubleType()),
    StructField("plant_height_90DAT_CM", DoubleType()),
    StructField("days_flower_initiation_DAYS", DoubleType()),
    StructField("nu_primery_branches_Num", DoubleType()),
    StructField("fruit_length_cm", DoubleType()),
    StructField("fruit_circumference_mm", DoubleType()),
    StructField("avg_fruit_weight_gram", DoubleType()),
    StructField("nu_fruits_per_plant", DoubleType()),
    StructField("fruit_TSS_Brix", DoubleType()),
    StructField("fruit_yield_per_plant_Kg", DoubleType()),
    StructField("yield_status", StringType()),
    StructField("morpho_details", ArrayType(StructType([
        StructField("StemAnthocyanin", StringType()),
        StructField("AnthocyaninIntensity", StringType()),
        StructField("StemPubescence", StringType()),
        StructField("StemPubescenceIntensity", StringType()),
        StructField("LeafBladeColour", StringType()),
        StructField("LeafBladeColourIntensity", StringType()),
        StructField("ColourOfVein", StringType()),
        StructField("IntensityOfColour8", StringType()),
        StructField("SpineonLeaf", StringType()),
        StructField("FlowerColour", StringType()),
        StructField("FruitingPattern", StringType()),
        StructField("FruitColour", StringType()),
        StructField("IntensityofColour13", StringType()),
        StructField("Stripes", StringType()),
        StructField("StripesDensity", StringType()),
        StructField("FruitPatches", StringType()),
        StructField("SpinesInCalyx", StringType()),
        StructField("DensityOfSpininess", StringType()),
        StructField("FruitShape", StringType())
    ])))
])


7. capcture the value data into json format 

In [ ]:
parsed_growth_df = desrialised_df.select("key", from_json("value", schema).alias("value"), "topic", "partition", "offset", "timestamp", "timestampType")
parsed_growth_df.createOrReplaceTempView("growth")

In [ ]:
%sql select * from growth

key,value,topic,partition,offset,timestamp,timestampType
101,"List(101, 47.47, 52.52, 63.63, 35.19, 11.39, 6.21, 12.63, 33.89, 6.06, 5.95, 203.34, Medium, List(List(Absent, Absent, Present, Medium, Green, Dark, Green, Medium, Absent, Purple, Cluster, Purple, Medium, Present, Sparse, Present, Absent, Absent, Obovate)))",growth_data_stream,2,0,2024-07-04T13:03:12.185+0000,0
120,"List(120, 72.1, 61.8, 74.16, 45.6, 6.72, 5.7, 11.31, 38.43, 6.18, 4.75, 230.58, Medium, List(List(Absent, Absent, Present, Medium, Green, Dark, Green, Medium, Absent, Dark _Purple, Cluster, Green, Medium, Absent, Absent, Present, Present, Med ium, Globular)))",growth_data_stream,2,1,2024-07-04T13:03:47.276+0000,0
121,"List(121, 70.7, 60.6, 72.72, 48.72, 7.18, 6.09, 12.08, 41.06, 6.06, 5.08, 246.36, Medium, List(List(Absent, Absent, Present, Strong, Green, Medium, Green, Medium, Absent, Light_Purple, Solitary, Purple, Dark, Absent, Absent, Absent, Present, Medium, Obovate)))",growth_data_stream,2,2,2024-07-04T13:03:47.896+0000,0
122,"List(122, 49.98, 52.02, 64.26, 53.82, 7.32, 5.8, 15.53, 58.37, 5.1, 4.4, 291.87, High, List(List(Present, Medium, Present, Medium, Green, Dark, Purple, Medium, Absent, Light_Purple, Solitary, Greenish_White, Light, Absent, Absent, Absent, Absent, Absent, Obovate)))",growth_data_stream,2,3,2024-07-04T13:03:48.522+0000,0
128,"List(128, 32.24, 33.28, 50.96, 38.3, 3.75, 6.0, 5.19, 46.06, 4.16, 5.18, 184.23, Low, List(List(Absent, Absent, Present, Weak, Green, Dark, Green, Light, Absent, Light_Purple, Cluster, Light_Green, Medium, Absent, Absent, Present, Absent, Absent, Globular)))",growth_data_stream,2,4,2024-07-04T13:03:52.264+0000,0
134,"List(134, 70.72, 58.24, 67.6, 44.51, 6.54, 6.11, 12.83, 31.58, 13.52, 3.88, 410.5, Very High, List(List(Absent, Absent, Present, Weak, Green, Light, Green, Light, Present, Light Purple, Solitary, Purple, Light, Present, Strong, Present, Present, Med ium, Globular)))",growth_data_stream,2,5,2024-07-04T13:03:55.819+0000,0
135,"List(135, 65.96, 54.32, 63.05, 40.85, 6.0, 5.61, 11.78, 28.98, 12.61, 3.56, 376.79, Very High, List(List(Absent, Absent, Present, Medium, Green, Light, Green, Light, Absent, Light Purple, Solitary, Green, Light, Absent, Absent, Absent, Absent, Absent, Ovoid)))",growth_data_stream,2,6,2024-07-04T13:03:56.340+0000,0
136,"List(136, 67.32, 55.44, 64.35, 43.65, 6.41, 5.99, 12.59, 30.97, 12.87, 3.81, 402.57, Very High, List(List(Present, Medium, Present, Medium, Green, Medium, Purple, Medium, Absent, Purple, Cluster, Purple, Dark, Absent, Absent, Absent, Absent, Absent, Club)))",growth_data_stream,2,7,2024-07-04T13:03:56.842+0000,0
137,"List(137, 73.84, 64.48, 65.52, 32.09, 5.85, 8.59, 8.38, 25.88, 6.24, 4.14, 155.25, Low, List(List(Present, Medium, Present, Weak, Green, Dark, Purple, Dark, Absent, Purple, M ixed, Purple, Medium, Present, Sparse, Present, Absent, Absent, Obovate)))",growth_data_stream,2,8,2024-07-04T13:03:57.342+0000,0
140,"List(140, 55.12, 50.96, 61.36, 45.54, 6.21, 8.59, 9.0, 19.3, 14.56, 4.92, 270.24, Medium, List(List(Present, Weak, Present, Medium, Green, Medium, Purple, Light, Absent, Greenish_White, Solitary, White, Dark, Absent, Absent, Absent, Absent, Absent, Globular)))",growth_data_stream,2,9,2024-07-04T13:03:59.086+0000,0


8a. explode your nested json data

In [ ]:
exploded_growth_df = spark.sql("""
          select key, value.GenotypeName as GenotypeName, 
          value.plant_height_30DAT_CM as plant_height_30DAT_CM, 
          value.plant_height_60DAT_CM as plant_height_60DAT_CM, 
          value.plant_height_90DAT_CM as plant_height_90DAT_CM, 
          value.days_flower_initiation_DAYS as days_flower_initiation_DAYS, 
          value.nu_primery_branches_Num as nu_primery_branches_Num, 
          value.fruit_length_cm as fruit_length_cm, 
          value.fruit_circumference_mm as fruit_circumference_mm, 
          value.avg_fruit_weight_gram as avg_fruit_weight_gram, 
          value.nu_fruits_per_plant as nu_fruits_per_plant, 
          value.fruit_TSS_Brix as fruit_TSS_Brix, 
          value.fruit_yield_per_plant_Kg as fruit_yield_per_plant_Kg, 
          value.yield_status as yield_status,
          explode (value.morpho_details) as morpho_details from growth
          """)
exploded_growth_df.createOrReplaceTempView("exploded_growth")

In [ ]:
%sql select * from exploded_growth

key,GenotypeName,plant_height_30DAT_CM,plant_height_60DAT_CM,plant_height_90DAT_CM,days_flower_initiation_DAYS,nu_primery_branches_Num,fruit_length_cm,fruit_circumference_mm,avg_fruit_weight_gram,nu_fruits_per_plant,fruit_TSS_Brix,fruit_yield_per_plant_Kg,yield_status,morpho_details
101,101,47.47,52.52,63.63,35.19,11.39,6.21,12.63,33.89,6.06,5.95,203.34,Medium,"List(Absent, Absent, Present, Medium, Green, Dark, Green, Medium, Absent, Purple, Cluster, Purple, Medium, Present, Sparse, Present, Absent, Absent, Obovate)"
120,120,72.1,61.8,74.16,45.6,6.72,5.7,11.31,38.43,6.18,4.75,230.58,Medium,"List(Absent, Absent, Present, Medium, Green, Dark, Green, Medium, Absent, Dark _Purple, Cluster, Green, Medium, Absent, Absent, Present, Present, Med ium, Globular)"
121,121,70.7,60.6,72.72,48.72,7.18,6.09,12.08,41.06,6.06,5.08,246.36,Medium,"List(Absent, Absent, Present, Strong, Green, Medium, Green, Medium, Absent, Light_Purple, Solitary, Purple, Dark, Absent, Absent, Absent, Present, Medium, Obovate)"
122,122,49.98,52.02,64.26,53.82,7.32,5.8,15.53,58.37,5.1,4.4,291.87,High,"List(Present, Medium, Present, Medium, Green, Dark, Purple, Medium, Absent, Light_Purple, Solitary, Greenish_White, Light, Absent, Absent, Absent, Absent, Absent, Obovate)"
128,128,32.24,33.28,50.96,38.3,3.75,6.0,5.19,46.06,4.16,5.18,184.23,Low,"List(Absent, Absent, Present, Weak, Green, Dark, Green, Light, Absent, Light_Purple, Cluster, Light_Green, Medium, Absent, Absent, Present, Absent, Absent, Globular)"
134,134,70.72,58.24,67.6,44.51,6.54,6.11,12.83,31.58,13.52,3.88,410.5,Very High,"List(Absent, Absent, Present, Weak, Green, Light, Green, Light, Present, Light Purple, Solitary, Purple, Light, Present, Strong, Present, Present, Med ium, Globular)"
135,135,65.96,54.32,63.05,40.85,6.0,5.61,11.78,28.98,12.61,3.56,376.79,Very High,"List(Absent, Absent, Present, Medium, Green, Light, Green, Light, Absent, Light Purple, Solitary, Green, Light, Absent, Absent, Absent, Absent, Absent, Ovoid)"
136,136,67.32,55.44,64.35,43.65,6.41,5.99,12.59,30.97,12.87,3.81,402.57,Very High,"List(Present, Medium, Present, Medium, Green, Medium, Purple, Medium, Absent, Purple, Cluster, Purple, Dark, Absent, Absent, Absent, Absent, Absent, Club)"
137,137,73.84,64.48,65.52,32.09,5.85,8.59,8.38,25.88,6.24,4.14,155.25,Low,"List(Present, Medium, Present, Weak, Green, Dark, Purple, Dark, Absent, Purple, M ixed, Purple, Medium, Present, Sparse, Present, Absent, Absent, Obovate)"
140,140,55.12,50.96,61.36,45.54,6.21,8.59,9.0,19.3,14.56,4.92,270.24,Medium,"List(Present, Weak, Present, Medium, Green, Medium, Purple, Light, Absent, Greenish_White, Solitary, White, Dark, Absent, Absent, Absent, Absent, Absent, Globular)"


9. flattened the exploded data

In [ ]:
flatned_growth_df = spark.sql("""
    SELECT 
        key,
        GenotypeName,
        plant_height_30DAT_CM,
        plant_height_60DAT_CM,
        plant_height_90DAT_CM,
        days_flower_initiation_DAYS,
        nu_primery_branches_Num,
        fruit_circumference_mm,
        avg_fruit_weight_gram,
        nu_fruits_per_plant,
        fruit_TSS_Brix,
        fruit_yield_per_plant_Kg,
        yield_status,
        morpho_details.StemAnthocyanin AS StemAnthocyanin,
        morpho_details.AnthocyaninIntensity AS AnthocyaninIntensity,
        morpho_details.StemPubescence AS StemPubescence,
        morpho_details.StemPubescenceIntensity AS StemPubescenceIntensity,
        morpho_details.LeafBladeColour AS LeafBladeColour,
        morpho_details.LeafBladeColourIntensity AS LeafBladeColourIntensity,
        morpho_details.ColourOfVein AS ColourOfVein,
        morpho_details.IntensityOfColour8 AS IntensityOfColour8,
        morpho_details.SpineonLeaf AS SpineonLeaf,
        morpho_details.FlowerColour AS FlowerColour,
        morpho_details.FruitingPattern AS FruitingPattern,
        morpho_details.FruitColour AS FruitColour,
        morpho_details.IntensityofColour13 AS IntensityofColour13,
        morpho_details.Stripes AS Stripes,
        morpho_details.StripesDensity AS StripesDensity,
        morpho_details.FruitPatches AS FruitPatches,
        morpho_details.SpinesInCalyx AS SpinesInCalyx,
        morpho_details.DensityOfSpininess AS DensityOfSpininess,
        morpho_details.FruitShape AS FruitShape
    FROM exploded_growth
""")

flatned_growth_df.createOrReplaceTempView("final_kafka_reads")


In [ ]:
%sql 
select * from final_kafka_reads

key,GenotypeName,plant_height_30DAT_CM,plant_height_60DAT_CM,plant_height_90DAT_CM,days_flower_initiation_DAYS,nu_primery_branches_Num,fruit_circumference_mm,avg_fruit_weight_gram,nu_fruits_per_plant,fruit_TSS_Brix,fruit_yield_per_plant_Kg,yield_status,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape
101,101,47.47,52.52,63.63,35.19,11.39,12.63,33.89,6.06,5.95,203.34,Medium,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate
120,120,72.1,61.8,74.16,45.6,6.72,11.31,38.43,6.18,4.75,230.58,Medium,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Dark _Purple,Cluster,Green,Medium,Absent,Absent,Present,Present,Med ium,Globular
121,121,70.7,60.6,72.72,48.72,7.18,12.08,41.06,6.06,5.08,246.36,Medium,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Present,Medium,Obovate
122,122,49.98,52.02,64.26,53.82,7.32,15.53,58.37,5.1,4.4,291.87,High,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Greenish_White,Light,Absent,Absent,Absent,Absent,Absent,Obovate
128,128,32.24,33.28,50.96,38.3,3.75,5.19,46.06,4.16,5.18,184.23,Low,Absent,Absent,Present,Weak,Green,Dark,Green,Light,Absent,Light_Purple,Cluster,Light_Green,Medium,Absent,Absent,Present,Absent,Absent,Globular
134,134,70.72,58.24,67.6,44.51,6.54,12.83,31.58,13.52,3.88,410.5,Very High,Absent,Absent,Present,Weak,Green,Light,Green,Light,Present,Light Purple,Solitary,Purple,Light,Present,Strong,Present,Present,Med ium,Globular
135,135,65.96,54.32,63.05,40.85,6.0,11.78,28.98,12.61,3.56,376.79,Very High,Absent,Absent,Present,Medium,Green,Light,Green,Light,Absent,Light Purple,Solitary,Green,Light,Absent,Absent,Absent,Absent,Absent,Ovoid
136,136,67.32,55.44,64.35,43.65,6.41,12.59,30.97,12.87,3.81,402.57,Very High,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Cluster,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Club
137,137,73.84,64.48,65.52,32.09,5.85,8.38,25.88,6.24,4.14,155.25,Low,Present,Medium,Present,Weak,Green,Dark,Purple,Dark,Absent,Purple,M ixed,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate
140,140,55.12,50.96,61.36,45.54,6.21,9.0,19.3,14.56,4.92,270.24,Medium,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Greenish_White,Solitary,White,Dark,Absent,Absent,Absent,Absent,Absent,Globular


CASE_I WRITE YOUR KAFKA READ INTO DELTA TABLE TO HIVE METASTORE.

In [ ]:
write_df = flatned_growth_df.write \
    .format("delta") \
    .mode("append") \
    .option("checkpointLocation", "check_pt110") \
    .saveAsTable("default.orders_new110")

In [ ]:
%sql
select * from default.orders_new110

key,GenotypeName,plant_height_30DAT_CM,plant_height_60DAT_CM,plant_height_90DAT_CM,days_flower_initiation_DAYS,nu_primery_branches_Num,fruit_circumference_mm,avg_fruit_weight_gram,nu_fruits_per_plant,fruit_TSS_Brix,fruit_yield_per_plant_Kg,yield_status,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape
101,101,47.47,52.52,63.63,35.19,11.39,12.63,33.89,6.06,5.95,203.34,Medium,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Purple,Cluster,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate
120,120,72.1,61.8,74.16,45.6,6.72,11.31,38.43,6.18,4.75,230.58,Medium,Absent,Absent,Present,Medium,Green,Dark,Green,Medium,Absent,Dark _Purple,Cluster,Green,Medium,Absent,Absent,Present,Present,Med ium,Globular
121,121,70.7,60.6,72.72,48.72,7.18,12.08,41.06,6.06,5.08,246.36,Medium,Absent,Absent,Present,Strong,Green,Medium,Green,Medium,Absent,Light_Purple,Solitary,Purple,Dark,Absent,Absent,Absent,Present,Medium,Obovate
122,122,49.98,52.02,64.26,53.82,7.32,15.53,58.37,5.1,4.4,291.87,High,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Greenish_White,Light,Absent,Absent,Absent,Absent,Absent,Obovate
128,128,32.24,33.28,50.96,38.3,3.75,5.19,46.06,4.16,5.18,184.23,Low,Absent,Absent,Present,Weak,Green,Dark,Green,Light,Absent,Light_Purple,Cluster,Light_Green,Medium,Absent,Absent,Present,Absent,Absent,Globular
134,134,70.72,58.24,67.6,44.51,6.54,12.83,31.58,13.52,3.88,410.5,Very High,Absent,Absent,Present,Weak,Green,Light,Green,Light,Present,Light Purple,Solitary,Purple,Light,Present,Strong,Present,Present,Med ium,Globular
135,135,65.96,54.32,63.05,40.85,6.0,11.78,28.98,12.61,3.56,376.79,Very High,Absent,Absent,Present,Medium,Green,Light,Green,Light,Absent,Light Purple,Solitary,Green,Light,Absent,Absent,Absent,Absent,Absent,Ovoid
136,136,67.32,55.44,64.35,43.65,6.41,12.59,30.97,12.87,3.81,402.57,Very High,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Purple,Cluster,Purple,Dark,Absent,Absent,Absent,Absent,Absent,Club
137,137,73.84,64.48,65.52,32.09,5.85,8.38,25.88,6.24,4.14,155.25,Low,Present,Medium,Present,Weak,Green,Dark,Purple,Dark,Absent,Purple,M ixed,Purple,Medium,Present,Sparse,Present,Absent,Absent,Obovate
140,140,55.12,50.96,61.36,45.54,6.21,9.0,19.3,14.56,4.92,270.24,Medium,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Greenish_White,Solitary,White,Dark,Absent,Absent,Absent,Absent,Absent,Globular


CASE-II: WRITE BACK THE KAFKA READS TO NEW KAFKA TOPIC.

In [ ]:
    desrialised_df \
    .writeStream \
    .format("kafka") \
    .outputMode("append") \
    .option("checkpointLocation", "checkpointdir105") \
    .option("kafka.bootstrap.servers", "******************************") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='****************' password='******************************************';") \
    .option("kafka.ssl.endpoint.identification.algorithm", "https") \
    .option("topic", '**************') \
    .start()

Out[26]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f1c1fc10670>

CASE - III. BUILD A TWO TIER DELTA_ARCHITEECHTETURE (MADALLION_ARCHITECHTURE)

In [ ]:
%fs mkdirs "/FileStore/delta_archi_stream/"

res0: Boolean = true

In [ ]:
%sql
CREATE DATABASE delta_archi_stream;
USE delta_archi_stream;

1. create bronze layer table

In [ ]:
%sql
CREATE TABLE delta_archi_stream.plant_yield_stream_bronze(
        KEY INT,
        GenotypeName INT,
        plant_height_30DAT_CM DOUBLE,
        plant_height_60DAT_CM DOUBLE,
        plant_height_90DAT_CM DOUBLE,
        days_flower_initiation_DAYS DOUBLE,
        nu_primery_branches_Num DOUBLE,
        fruit_circumference_mm DOUBLE,
        avg_fruit_weight_gram DOUBLE,
        nu_fruits_per_plant DOUBLE,
        fruit_TSS_Brix DOUBLE,
        fruit_yield_per_plant_Kg DOUBLE,
        yield_status STRING,
        StemAnthocyanin STRING,
        AnthocyaninIntensity STRING,
        StemPubescence STRING,
        StemPubescenceIntensity STRING,
        LeafBladeColour STRING,
        LeafBladeColourIntensity STRING,
        ColourOfVein STRING,
        IntensityOfColour8 STRING,
        SpineonLeaf STRING,
        FlowerColour STRING,
        FruitingPattern STRING,
        FruitColour STRING,
        IntensityofColour13 STRING,
        Stripes STRING,
        StripesDensity STRING,
        FruitPatches STRING,
        SpinesInCalyx STRING,
        DensityOfSpininess STRING,
        FruitShape STRING
)
USING DELTA
LOCATION "dbfs:/FileStore/delta_archi_stream/plant_yield_stream_bronze.delta"
PARTITIONED BY (yield_status)
TBLPROPERTIES (delta.enableChangeDataFeed = true);

2. Create Silver Layer Table

In [ ]:
%sql
CREATE TABLE delta_archi_stream.plant_yield_stream_silver(
        KEY INT,
        GenotypeName INT,
        average_plant_hieght_CM DOUBLE,
        fruit_circumference_mm DOUBLE,
        avg_fruit_weight_gram DOUBLE,
        fruit_TSS_Brix DOUBLE,
        fruit_yield_per_plant_Kg DOUBLE,
        yield_status STRING,
        FruitAnthocyaninIntensity STRING,
        StemPubescenceIntensity STRING,
        LeafBladeColourIntensity STRING,
        IntensityOfColour STRING,
        SpineonLeaf STRING,
        FlowerColour STRING,
        FruitingPattern STRING,
        FruitIntensityofColour STRING,
        StripesDensity STRING,
        FruitPatches STRING,
        DensityOfSpininessOnCalyx STRING,
        FruitShape STRING
)
USING DELTA
LOCATION "dbfs:/FileStore/delta_archi_stream/plant_yield_stream_silver.delta"
PARTITIONED BY (yield_status)
TBLPROPERTIES (delta.enableChangeDataFeed = true);

3. Migration of data write stream delta table to bronze layer delta table
Use merge command to  mentain idempotency of table data

In [ ]:

%sql
MERGE INTO delta_archi_stream.plant_yield_stream_bronze AS target
USING default.orders_new110 AS source
ON target.KEY = source.KEY
WHEN MATCHED THEN
  UPDATE SET
    target.GenotypeName = source.GenotypeName,
    target.plant_height_30DAT_CM = source.plant_height_30DAT_CM,
    target.plant_height_60DAT_CM = source.plant_height_60DAT_CM,
    target.plant_height_90DAT_CM = source.plant_height_90DAT_CM,
    target.days_flower_initiation_DAYS = source.days_flower_initiation_DAYS,
    target.nu_primery_branches_Num = source.nu_primery_branches_Num,
    target.fruit_circumference_mm = source.fruit_circumference_mm,
    target.avg_fruit_weight_gram = source.avg_fruit_weight_gram,
    target.nu_fruits_per_plant = source.nu_fruits_per_plant,
    target.fruit_TSS_Brix = source.fruit_TSS_Brix,
    target.fruit_yield_per_plant_Kg = source.fruit_yield_per_plant_Kg,
    target.yield_status = source.yield_status,
    target.StemAnthocyanin = source.StemAnthocyanin,
    target.AnthocyaninIntensity = source.AnthocyaninIntensity,
    target.StemPubescence = source.StemPubescence,
    target.StemPubescenceIntensity = source.StemPubescenceIntensity,
    target.LeafBladeColour = source.LeafBladeColour,
    target.LeafBladeColourIntensity = source.LeafBladeColourIntensity,
    target.ColourOfVein = source.ColourOfVein,
    target.IntensityOfColour8 = source.IntensityOfColour8,
    target.SpineonLeaf = source.SpineonLeaf,
    target.FlowerColour = source.FlowerColour,
    target.FruitingPattern = source.FruitingPattern,
    target.FruitColour = source.FruitColour,
    target.IntensityofColour13 = source.IntensityofColour13,
    target.Stripes = source.Stripes,
    target.StripesDensity = source.StripesDensity,
    target.FruitPatches = source.FruitPatches,
    target.SpinesInCalyx = source.SpinesInCalyx,
    target.DensityOfSpininess = source.DensityOfSpininess,
    target.FruitShape = source.FruitShape
WHEN NOT MATCHED THEN
  INSERT (
    KEY,
    GenotypeName,
    plant_height_30DAT_CM,
    plant_height_60DAT_CM,
    plant_height_90DAT_CM,
    days_flower_initiation_DAYS,
    nu_primery_branches_Num,
    fruit_circumference_mm,
    avg_fruit_weight_gram,
    nu_fruits_per_plant,
    fruit_TSS_Brix,
    fruit_yield_per_plant_Kg,
    yield_status,
    StemAnthocyanin,
    AnthocyaninIntensity,
    StemPubescence,
    StemPubescenceIntensity,
    LeafBladeColour,
    LeafBladeColourIntensity,
    ColourOfVein,
    IntensityOfColour8,
    SpineonLeaf,
    FlowerColour,
    FruitingPattern,
    FruitColour,
    IntensityofColour13,
    Stripes,
    StripesDensity,
    FruitPatches,
    SpinesInCalyx,
    DensityOfSpininess,
    FruitShape
  ) VALUES (
    source.KEY,
    source.GenotypeName,
    source.plant_height_30DAT_CM,
    source.plant_height_60DAT_CM,
    source.plant_height_90DAT_CM,
    source.days_flower_initiation_DAYS,
    source.nu_primery_branches_Num,
    source.fruit_circumference_mm,
    source.avg_fruit_weight_gram,
    source.nu_fruits_per_plant,
    source.fruit_TSS_Brix,
    source.fruit_yield_per_plant_Kg,
    source.yield_status,
    source.StemAnthocyanin,
    source.AnthocyaninIntensity,
    source.StemPubescence,
    source.StemPubescenceIntensity,
    source.LeafBladeColour,
    source.LeafBladeColourIntensity,
    source.ColourOfVein,
    source.IntensityOfColour8,
    source.SpineonLeaf,
    source.FlowerColour,
    source.FruitingPattern,
    source.FruitColour,
    source.IntensityofColour13,
    source.Stripes,
    source.StripesDensity,
    source.FruitPatches,
    source.SpinesInCalyx,
    source.DensityOfSpininess,
    source.FruitShape
  )


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
300,0,0,300


In [ ]:
%sql
select * from delta_archi_stream.plant_yield_stream_bronze

KEY,GenotypeName,plant_height_30DAT_CM,plant_height_60DAT_CM,plant_height_90DAT_CM,days_flower_initiation_DAYS,nu_primery_branches_Num,fruit_circumference_mm,avg_fruit_weight_gram,nu_fruits_per_plant,fruit_TSS_Brix,fruit_yield_per_plant_Kg,yield_status,StemAnthocyanin,AnthocyaninIntensity,StemPubescence,StemPubescenceIntensity,LeafBladeColour,LeafBladeColourIntensity,ColourOfVein,IntensityOfColour8,SpineonLeaf,FlowerColour,FruitingPattern,FruitColour,IntensityofColour13,Stripes,StripesDensity,FruitPatches,SpinesInCalyx,DensityOfSpininess,FruitShape
144,144,60.14,74.69,77.6,44.65,4.75,8.27,27.23,5.82,5.23,163.4,Low,Absent,Absent,Present,Medium,Green,Dark,Green,Light,Absent,Purple,Mixed,Green,Medium,Absent,Absent,Present,Present,Medium,Obovate
146,146,56.16,43.68,50.96,41.4,6.21,10.35,31.05,5.2,4.92,155.25,Low,Absent,Absent,Present,Medium,Green,Light,Green,Light,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Present,Absent,Absent,Obovate
170,170,31.2,58.24,69.68,48.65,3.5,20.18,73.49,2.08,5.18,146.97,Low,Present,Weak,Present,Medium,Green,Medium,Green,Light,Present,Purple,Solitary,Green,Medium,Absent,Absent,Absent,Present,Strong,Club
172,172,29.7,55.44,66.33,47.71,3.43,19.79,72.07,1.98,5.08,144.13,Low,Present,Medium,Present,Medium,Green,Medium,Purple,Medium,Absent,Light_Purple,Solitary,Purple,Light,Absent,Absent,Present,Absent,Absent,Ovoid
222,222,56.26,47.53,57.23,38.0,5.7,12.26,25.81,5.82,4.04,154.85,Low,Present,Weak,Present,Medium,Green,Medium,Purple,Light,Absent,Greenish_White,Solitary,White,Dark,Absent,Absent,Absent,Absent,Absent,Globular
236,236,55.12,62.4,74.88,41.4,5.85,17.08,45.47,4.16,3.62,181.87,Low,Present,Medium,Present,Medium,Green,Dark,Purple,Medium,Absent,Light_Purple,Solitary,Greenish_White,Light,Absent,Absent,Absent,Absent,Absent,Obovate
237,237,51.41,58.2,69.84,38.0,5.37,15.68,41.73,3.88,3.33,166.93,Low,Present,Medium,Present,Strong,Green,Dark,Purple,Dark,Absent,Purple,Solitary,Purple,Medium,Absent,Absent,Absent,Absent,Absent,Ovoid
144,144,60.14,74.69,77.6,44.65,4.75,8.27,27.23,5.82,5.23,163.4,Low,Absent,Absent,Present,Medium,Green,Dark,Green,Light,Absent,Purple,Mixed,Green,Medium,Absent,Absent,Present,Present,Medium,Obovate
146,146,56.16,43.68,50.96,41.4,6.21,10.35,31.05,5.2,4.92,155.25,Low,Absent,Absent,Present,Medium,Green,Light,Green,Light,Absent,Purple,Solitary,Purple,Medium,Present,Strong,Present,Absent,Absent,Obovate
170,170,31.2,58.24,69.68,48.65,3.5,20.18,73.49,2.08,5.18,146.97,Low,Present,Weak,Present,Medium,Green,Medium,Green,Light,Present,Purple,Solitary,Green,Medium,Absent,Absent,Absent,Present,Strong,Club


4. Migration of data write stream delta table to SILVER layer delta table
Use OVERWRITE command to  FINALISE data with aggregates, crunching of table data

In [ ]:
%sql
INSERT OVERWRITE TABLE delta_archi_stream.plant_yield_stream_silver
SELECT
  KEY,
  GenotypeName,
  avg((plant_height_30DAT_CM + plant_height_60DAT_CM + plant_height_90DAT_CM)/3) AS average_plant_height_CM,
  fruit_circumference_mm, 
  avg_fruit_weight_gram,
  fruit_TSS_Brix,
  fruit_yield_per_plant_Kg,
  yield_status,
  AnthocyaninIntensity AS FruitAnthocyaninIntensity,
  StemPubescenceIntensity AS StemPubescenceIntensity,
  LeafBladeColourIntensity AS LeafBladeColourIntensity,
  IntensityOfColour8 AS IntensityOfColour,
  SpineonLeaf AS SpineonLeaf,
  FlowerColour AS FlowerColour,
  FruitingPattern AS FruitingPattern,
  IntensityofColour13 AS FruitIntensityofColour,
  StripesDensity AS StripesDensity,
  FruitPatches AS FruitPatches,
  DensityOfSpininess AS DensityOfSpininessOnCalyx,
  FruitShape AS FruitShape
FROM delta_archi_stream.plant_yield_stream_bronze
GROUP BY 
  KEY,
  GenotypeName,
  fruit_circumference_mm, 
  avg_fruit_weight_gram,
  fruit_TSS_Brix,
  fruit_yield_per_plant_Kg,
  yield_status,
  AnthocyaninIntensity,
  StemPubescenceIntensity,
  LeafBladeColourIntensity,
  IntensityOfColour8,
  SpineonLeaf,
  FlowerColour,
  FruitingPattern,
  IntensityofColour13,
  StripesDensity,
  FruitPatches,
  DensityOfSpininess,
  FruitShape
ORDER BY GenotypeName


num_affected_rows,num_inserted_rows
150,150


In [ ]:
%sql
select * from delta_archi_stream.plant_yield_stream_silver

KEY,GenotypeName,average_plant_hieght_CM,fruit_circumference_mm,avg_fruit_weight_gram,fruit_TSS_Brix,fruit_yield_per_plant_Kg,yield_status,FruitAnthocyaninIntensity,StemPubescenceIntensity,LeafBladeColourIntensity,IntensityOfColour,SpineonLeaf,FlowerColour,FruitingPattern,FruitIntensityofColour,StripesDensity,FruitPatches,DensityOfSpininessOnCalyx,FruitShape
128,128,38.826666666666675,5.19,46.06,5.18,184.23,Low,Absent,Weak,Dark,Light,Absent,Light_Purple,Cluster,Medium,Absent,Present,Absent,Globular
129,129,36.21333333333333,4.76,42.28,4.75,169.1,Low,Absent,Medium,Dark,Light,Absent,Purple,Solitary,Light,Absent,Absent,Absent,Ovoid
130,130,36.96,5.09,45.17,5.08,180.67,Low,Absent,Strong,Medium,Medium,Absent,Purple,Solitary,Light,Absent,Absent,Absent,Globular
137,137,67.94666666666666,8.38,25.88,4.14,155.25,Low,Medium,Weak,Dark,Dark,Absent,Purple,M ixed,Medium,Sparse,Present,Absent,Obovate
138,138,63.373333333333335,7.7,23.75,3.8,142.5,Low,Absent,Medium,Dark,Medium,Absent,Purple,Cluster,Medium,Absent,Present,Med ium,Obovate
139,139,64.68,8.22,25.38,4.06,152.25,Low,Absent,Weak,Light,Light,Absent,Purple,Solitary,Medium,Absent,Present,Absent,Ovoid
143,143,75.92,9.0,29.67,5.69,178.02,Low,Medium,Medium,Dark,Medium,Absent,Light_Purple,Solitary,Dark,Absent,Absent,Absent,Globular
144,144,70.80999999999999,8.27,27.23,5.23,163.4,Low,Absent,Medium,Dark,Light,Absent,Purple,Mixed,Medium,Absent,Present,Medium,Obovate
145,145,72.27,8.83,29.1,5.58,174.58,Low,Absent,Weak,Light,Light,Absent,Light_Purple,Cluster,Dark,Absent,Absent,Absent,Obovate
146,146,50.26666666666667,10.35,31.05,4.92,155.25,Low,Absent,Medium,Light,Light,Absent,Purple,Solitary,Medium,Strong,Present,Absent,Obovate
